In [`02_preprocessing.ipynb`](02_preprocessing.ipynb) hebben we één werkbaar `.npy`-bestand gemaakt, waarbij elke rij overeenkomt met één pixel en alle bijbehorende features. Deze features bevatten de genormaliseerde RGB-waarden en de lokale textuur per kleurband.

In de volgende stappen ga ik aan de slag met KMeans clustering. Het doel hiervan is om groepen van vergelijkbare pixels te ontdekken zonder vooraf opgegeven labels. Elke groep (cluster) krijgt een uniek label, waarmee later bijvoorbeeld vegetatie, water, en verharding kan worden onderscheiden.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import time
import joblib 
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from fiona import Env
import os
import rasterio
from tqdm import tqdm
from pathlib import Path
from shapely.geometry import MultiPoint
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from rasterio.transform import from_origin, rowcol
from scipy.ndimage import generic_filter
from rasterio.features import geometry_mask

Eerst maak ik een handmatig gelabelde trainingset voor elke klasse oppervlakken. Vervolgens train ik een Random Forest classifier op de bestaande features, en dan pas ik het model toe op het hele raster. Ik hoop dat dat een beter resultaat geeft.

In [ ]:
# Padinstellingen
features_path = "../data/processed/features_all.npy"
shapefile_path = "../qgis/training_samples.shp"
output_dir = "../data/processed/training_batches"
os.makedirs(output_dir, exist_ok=True)

# Batch parameters
batch_size = 1_000_000

# Laad gelabelde polygonen
gdf = gpd.read_file(shapefile_path)
if gdf.crs.to_epsg() != 28992:
    gdf = gdf.to_crs(epsg=28992)

# Laad features (memory-mapped)
features = np.load(features_path, mmap_mode="r")
n_rows = features.shape[0]

# Start batchverwerking
sample_counter = 0
for i in tqdm(range(0, n_rows, batch_size), desc="Verwerken van batches"):
    end = min(i + batch_size, n_rows)
    batch = features[i:end]
    coords = batch[:, 6:8]  # RD X, Y

    # Maak GeoDataFrame van punten
    df_points = pd.DataFrame(coords, columns=["x", "y"])
    gdf_points = gpd.GeoDataFrame(
        df_points,
        geometry=gpd.points_from_xy(df_points.x, df_points.y),
        crs="EPSG:28992"
    )

    # Spatial join met polygonen
    joined = gpd.sjoin(gdf_points, gdf[["geometry", "label"]], how="inner", predicate="within")

    if not joined.empty:
        # Haal indexen en labels op
        matched_indices = joined.index.to_numpy()
        labels = joined["label"].to_numpy()

        # Haal features op (RGB + textuur)
        X_arr = batch[matched_indices, :6]
        y_arr = labels

        # Sla op als npy-bestanden
        np.save(os.path.join(output_dir, f"train_X_part_{sample_counter:03d}.npy"), X_arr)
        np.save(os.path.join(output_dir, f"train_y_part_{sample_counter:03d}.npy"), y_arr)
        print(f"Batch {i//batch_size + 1}: {len(X_arr)} gelabelde pixels opgeslagen")

        sample_counter += 1


Samenvoegen

In [ ]:
# Pad naar batchfolder
input_dir = "../data/processed/training_batches"
X_files = sorted(glob.glob(os.path.join(input_dir, "train_X_part_*.npy")))
y_files = sorted(glob.glob(os.path.join(input_dir, "train_y_part_*.npy")))

# Samenvoegen
X_all = np.vstack([np.load(f) for f in X_files])
y_all = np.concatenate([np.load(f) for f in y_files])

# Opslaan
np.save("../data/processed/train_X.npy", X_all)
np.save("../data/processed/train_y.npy", y_all)

print(f"Samengevoegd: {X_all.shape[0]} samples, {X_all.shape[1]} features.")


Samengevoegd: 7925475 samples, 6 features.


Modeltraining: Onverhard / Verhard (Water komt als overlay uit extern masker)

In [ ]:
# Labelnamen (alleen klassen 1 en 2 worden getraind — 3 = water, overlay)
label_names = {
    1: "Groen",
    2: "Verhard"
}

# Data laden
X = np.load("../data/processed/train_X.npy")
y = np.load("../data/processed/train_y.npy").astype(int)

# Filter op alleen klassen 1 en 2 (3 = water → komt extern uit watermasker)
valid_mask = np.isin(y, [1, 2])
X = X[valid_mask]
y = y[valid_mask]

# Train/test split (20% testset, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Random Forest model instellen
clf = RandomForestClassifier(
    n_estimators=250,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# Train model
print("Model trainen op groen/verhard...")
clf.fit(X_train, y_train)

# Voorspellen
y_pred = clf.predict(X_test)

# Evaluatie
print("\n Classification report:\n")
print(classification_report(
    y_test, y_pred, 
    target_names=[label_names[i] for i in sorted(label_names)]
))

print("\n Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

# Model opslaan
joblib.dump(clf, "../data/processed/random_forest_model.pkl")
print("\n Model opgeslagen als: random_forest_model.pkl")




Model trainen op groen/verhard...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 24.5min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    8.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:   10.4s finished



 Classification report:

              precision    recall  f1-score   support

       Groen       1.00      1.00      1.00    968765
     Verhard       1.00      0.99      1.00    616330

    accuracy                           1.00   1585095
   macro avg       1.00      1.00      1.00   1585095
weighted avg       1.00      1.00      1.00   1585095


 Confusion matrix:
[[966849   1916]
 [  3104 613226]]

 Model opgeslagen als: random_forest_model.pkl


Toepassen op Delftwijk met smoothing (majority vote).

In [ ]:
# Padinstellingen
features_path = "../data/processed/features_all.npy"
model_path = "../data/processed/random_forest_model.pkl"
wijken_path = "../qgis/wijken_haarlem.gpkg"
output_path = "../output/figures/delftwijk_smoothed.tif"
water_mask_path = "../qgis/water_mask.tif"
pixel_size = 0.25  # meters

# Model en data inladen
print("Model laden")
model = joblib.load(model_path)
features = np.load(features_path, mmap_mode='r')
X_all = features[:, :6]        # RGB + textuur
coords = features[:, 6:8]      # RD X, Y

# Wijk ophalen
print("Delftwijk ophalen")
wijken = gpd.read_file(wijken_path).to_crs("EPSG:28992")
wijk_geom = wijken[wijken["wijknaam"] == "Delftwijk"].geometry.values[0]
bbox_minx, bbox_miny, bbox_maxx, bbox_maxy = wijk_geom.bounds

# Prefilteren op bounding box in batches
print("Prefilteren in batches")
batch_size = 10_000_000
n = coords.shape[0]
selected_indices = []

for i in tqdm(range(0, n, batch_size), desc="Prefilter"):
    end = min(i + batch_size, n)
    x_batch = coords[i:end, 0]
    y_batch = coords[i:end, 1]
    mask = (
        (x_batch >= bbox_minx) & (x_batch <= bbox_maxx) &
        (y_batch >= bbox_miny) & (y_batch <= bbox_maxy)
    )
    matched = np.where(mask)[0] + i
    selected_indices.extend(matched)

selected_indices = np.array(selected_indices)
X_bbox = X_all[selected_indices]
coords = coords[selected_indices]
print(f"Geselecteerd: {len(coords):,} pixels in bounding box")

# Puntgeometrieën maken voor spatial join
print("Puntgeometrieën maken")
df_coords = pd.DataFrame(coords, columns=["x", "y"])
gdf_points = gpd.GeoDataFrame(df_coords, geometry=gpd.points_from_xy(df_coords["x"], df_coords["y"]), crs="EPSG:28992")

# Spatial join met Delftwijk
print("Spatial join met Delftwijk")
gdf_wijk = gpd.GeoDataFrame(geometry=[wijk_geom], crs="EPSG:28992")
matched = gpd.sjoin(gdf_points, gdf_wijk, how="inner", predicate="within")
final_indices = matched.index.to_numpy()
X_sub = X_bbox[final_indices]
coords_sub = coords[final_indices]

# Voorspellen
print("Classificatie uitvoeren")
labels = model.predict(X_sub)

# Water-overlay toepassen
print("Water-overlay toepassen")
with rasterio.open(water_mask_path) as water_src:
    water_data = water_src.read(1)
    water_transform = water_src.transform

rows, cols = rowcol(water_transform, coords_sub[:, 0], coords_sub[:, 1])
rows = np.clip(rows, 0, water_data.shape[0] - 1)
cols = np.clip(cols, 0, water_data.shape[1] - 1)
water_values = water_data[rows, cols]

# Overschrijf waterpixels met klasse 3
labels[water_values == 3] = 3

# Rasterformaat bepalen
x_min, y_min = coords_sub.min(axis=0)
x_max, y_max = coords_sub.max(axis=0)
width = int(np.ceil((x_max - x_min) / pixel_size))
height = int(np.ceil((y_max - y_min) / pixel_size))
transform = from_origin(x_min, y_max, pixel_size, pixel_size)

# Raster vullen
print("Raster vullen")
band = np.full((height, width), fill_value=255, dtype="uint8")  # 255 = NoData
x_idx = ((coords_sub[:, 0] - x_min) / pixel_size).astype(int).clip(0, width - 1)
y_idx = ((y_max - coords_sub[:, 1]) / pixel_size).astype(int).clip(0, height - 1)
band[y_idx, x_idx] = labels

# Smoothing toepassen (3x3 majority vote)
def mode_filter(arr):
    vals, counts = np.unique(arr[arr != 255], return_counts=True)
    return vals[np.argmax(counts)] if len(vals) > 0 else 255

print("Smoothing (3×3 majority filter)")
smoothed = generic_filter(band, mode_filter, size=3)

# Exporteren
print(f"Wegschrijven naar: {output_path}")
with rasterio.open(
    output_path, 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype="uint8",
    crs="EPSG:28992",
    transform=transform,
    nodata=255,
    compress="DEFLATE",
    predictor=2,
    zlevel=9,
    BIGTIFF="YES"
) as dst:
    dst.write(smoothed, 1)

print(f"Klaar! Gesmoothte raster opgeslagen als:\n{output_path}")



📦 Model laden...
📍 Delftwijk ophalen...
🔍 Prefilteren in batches...


Prefilter: 100%|██████████| 52/52 [04:53<00:00,  5.64s/it]


➡️ Geselecteerd: 14,442,888 pixels in bounding box
📍 Puntgeometrieën maken...
📌 Spatial join met Delftwijk...
🤖 Classificatie uitvoeren...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    9.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   32.3s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:   42.5s finished


💧 Water-overlay toepassen...
🧱 Raster vullen...
🔄 Smoothing (3×3 majority filter)...
💾 Wegschrijven naar: ../qgis/delftwijk_smoothed.tif

✅ Klaar! Gesmoothte raster opgeslagen als:
../qgis/delftwijk_smoothed.tif


In [ ]:
# Pad naar het resultaat
path = "../qgis/delftwijk_smoothed.tif"  # pas aan indien nodig
pixel_size = 0.25  # meter
pixel_area = pixel_size ** 2  # m2

with rasterio.open(path) as src:
    band = src.read(1)

# Filter op geldige pixels
valid_pixels = band[band != 255]

# Tel pixels per klasse
unique, counts = np.unique(valid_pixels, return_counts=True)
areas_m2 = counts * pixel_area
total_area = areas_m2.sum()

# Toon resultaat
print("Oppervlakteverdeling Delftwijk:")
for val, count, area in zip(unique, counts, areas_m2):
    label = {1: "Onverhard", 2: "Verhard", 3: "Water"}.get(val, f"Klasse {val}")
    pct = 100 * area / total_area
    print(f"• {label:10s}: {area:10.1f} m²  ({pct:.1f}%)")

print(f"\nTotale geclassificeerde oppervlakte: {total_area:,.1f} m²")


📊 Oppervlakteverdeling Delftwijk:
• Onverhard :   190213.8 m²  (39.2%)
• Verhard   :   279829.8 m²  (57.7%)
• Water     :    14932.6 m²  (3.1%)

Totale geclassificeerde oppervlakte: 484,976.2 m²


Analyse van Generaalsbuurt

In [ ]:
# Padinstellingen
features_path = "../data/processed/features_all.npy"
model_path = "../data/processed/random_forest_model.pkl"
buurten_path = "../qgis/buurten_haarlem.gpkg"
output_path = "../qgis/generaalsbuurt_smoothed.tif"
pixel_size = 0.25  # meters

# Data inladen
print("Data inladen")
features = np.load(features_path, mmap_mode="r")
X_all = features[:, :6]
coords = features[:, 6:8]
model = joblib.load(model_path)

# Generaalsbuurt ophalen
print("Buurt ophalen")
gdf = gpd.read_file(buurten_path).to_crs("EPSG:28992")
buurt_geom = gdf[gdf["buurtnaam"] == "Generaalsbuurt"].geometry.values[0]
bbox_minx, bbox_miny, bbox_maxx, bbox_maxy = buurt_geom.bounds

# Prefilteren op bounding box
print("Prefilter op bounding box")
batch_size = 10_000_000
n = coords.shape[0]
selected_indices = []

for i in tqdm(range(0, n, batch_size), desc="Prefilter"):
    end = min(i + batch_size, n)
    x_batch = coords[i:end, 0]
    y_batch = coords[i:end, 1]
    mask = (x_batch >= bbox_minx) & (x_batch <= bbox_maxx) & (y_batch >= bbox_miny) & (y_batch <= bbox_maxy)
    matched = np.where(mask)[0] + i
    selected_indices.extend(matched)

selected_indices = np.array(selected_indices)
X_bbox = X_all[selected_indices]
coords_bbox = coords[selected_indices]

# patial join met Generaalsbuurt
print("Spatial join")
df_coords = pd.DataFrame(coords_bbox, columns=["x", "y"])
gdf_points = gpd.GeoDataFrame(df_coords, geometry=gpd.points_from_xy(df_coords["x"], df_coords["y"]), crs="EPSG:28992")
gdf_buurt = gpd.GeoDataFrame(geometry=[buurt_geom], crs="EPSG:28992")
matched = gpd.sjoin(gdf_points, gdf_buurt, how="inner", predicate="within")
final_indices = matched.index.to_numpy()

X_sub = X_bbox[final_indices]
coords_sub = coords_bbox[final_indices]

# Classificatie
print("Classificatie uitvoeren")
labels = model.predict(X_sub)

# Rasterformaat bepalen
x_min, y_min = coords_sub.min(axis=0)
x_max, y_max = coords_sub.max(axis=0)
width = int(np.ceil((x_max - x_min) / pixel_size))
height = int(np.ceil((y_max - y_min) / pixel_size))
transform = from_origin(x_min, y_max, pixel_size, pixel_size)

# Raster vullen
print("Raster vullen")
band = np.full((height, width), fill_value=255, dtype="uint8")  # 255 = NoData
x_idx = ((coords_sub[:, 0] - x_min) / pixel_size).astype(int).clip(0, width - 1)
y_idx = ((y_max - coords_sub[:, 1]) / pixel_size).astype(int).clip(0, height - 1)
band[y_idx, x_idx] = labels

# Smoothing toepassen
def mode_filter(arr):
    vals, counts = np.unique(arr[arr != 255], return_counts=True)
    return vals[np.argmax(counts)] if len(vals) > 0 else 255

print("Smoothing toepassen")
smoothed = generic_filter(band, mode_filter, size=3)

# Exporteren als raster
print(f"Wegschrijven naar: {output_path}")
with rasterio.open(
    output_path,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype="uint8",
    crs="EPSG:28992",
    transform=transform,
    nodata=255,
    compress="DEFLATE",
    predictor=2,
    zlevel=9,
    BIGTIFF="YES"
) as dst:
    dst.write(smoothed, 1)

📦 Data inladen...
📍 Buurt ophalen...
🔍 Prefilter op bounding box...


Prefilter: 100%|██████████| 52/52 [04:49<00:00,  5.56s/it]


🤝 Spatial join...
🤖 Classificatie uitvoeren...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    8.6s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:   10.9s finished


🧱 Raster vullen...
🌀 Smoothing toepassen...
💾 Wegschrijven naar: ../qgis/generaalsbuurt_smoothed.tif

📊 Oppervlakteverdeling Generaalsbuurt:
• Onverhard :    15477.7 m²  (14.7%)
• Verhard   :    89602.7 m²  (85.3%)

🌱 Groen per woning: 13.70 m²


Volledige grid opsplitsen in batches, classificeren, en raster bouwen.

In [ ]:
# Padinstellingen
features_path = "../data/processed/features_all.npy"
model_path = "../data/processed/random_forest_model.pkl"
output_path = "../data/processed/haarlem_classified.tif"

pixel_size = 0.25  # meter
batch_size = 1_000_000  # aantal pixels per batch

# Laad model en features
print("Data en model laden")
model = joblib.load(model_path)
features = np.load(features_path, mmap_mode="r")
X_all = features[:, :6]
coords = features[:, 6:8]
n_pixels = X_all.shape[0]

# Rastergrootte bepalen
x_min, y_min = coords.min(axis=0)
x_max, y_max = coords.max(axis=0)
width = int(np.ceil((x_max - x_min) / pixel_size))
height = int(np.ceil((y_max - y_min) / pixel_size))
transform = from_origin(x_min, y_max, pixel_size, pixel_size)

# Lege rasterband aanmaken
print("Raster initialiseren")
band = np.full((height, width), 255, dtype="uint8")  # 255 = NoData

# Classificatie per batch
print("Classificeren in batches")
for i in tqdm(range(0, n_pixels, batch_size)):
    end = min(i + batch_size, n_pixels)
    X_batch = X_all[i:end]
    coords_batch = coords[i:end]
    labels = model.predict(X_batch)

    # Pixelindex in raster bepalen
    x_idx = ((coords_batch[:, 0] - x_min) / pixel_size).astype(int)
    y_idx = ((y_max - coords_batch[:, 1]) / pixel_size).astype(int)
    
    valid = (x_idx >= 0) & (x_idx < width) & (y_idx >= 0) & (y_idx < height)
    band[y_idx[valid], x_idx[valid]] = labels[valid]

# Wegschrijven naar raster
print(f"Wegschrijven naar {output_path}")
with rasterio.open(
    output_path,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype="uint8",
    crs="EPSG:28992",
    transform=transform,
    nodata=255,
    compress="DEFLATE",
    predictor=2,
    zlevel=9,
    BIGTIFF="YES"
) as dst:
    dst.write(band, 1)

print("Klaar: volledig grid geclassificeerd.")

Data en model laden...
Raster initialiseren...
Classificeren in batches...


  0%|          | 0/514 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    4.8s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    6.5s finished
  0%|          | 1/514 [00:07<1:05:35,  7.67s/it][Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    3.6s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    5.0s finished
  0%|          | 2/514 [00:13<56:02,  6.57s/it]  [Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    4.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    5.3s finished
  1%|          | 3/514 [00:19<53:53,  6.33s/it]

Wegschrijven naar ../data/processed/haarlem_classified.tif
Klaar: volledig grid geclassificeerd.


Smoothen raster (majority filter op gebied van 3x3 pixels). 

In [ ]:
import numpy as np
import rasterio
from rasterio.windows import Window
from scipy.ndimage import generic_filter

from tqdm import tqdm

# Instellingen
input_path = "../qgis/haarlem_geclassificeerd.tif"
output_path = "../qgis/haarlem_geclassificeerd_smoothed.tif"
nodata_value = 255
kernel_size = 3
pad = kernel_size // 2
batch_height = 1024  # aantal rijen per batch

# Majority filter functie
def mode_filter(arr):
    vals, counts = np.unique(arr[arr != nodata_value], return_counts=True)
    return vals[np.argmax(counts)] if len(vals) > 0 else nodata_value

# Open raster
print("Raster openen")
with rasterio.open(input_path) as src:
    profile = src.profile.copy()
    width, height = src.width, src.height

    profile.update({
        "compress": "DEFLATE",
        "predictor": 2,
        "zlevel": 9,
        "dtype": "uint8",
        "nodata": nodata_value
    })

    # Output aanmaken
    with rasterio.open(output_path, "w", **profile) as dst:
        n_batches = int(np.ceil(height / batch_height))
        total_time = 0

        print(f"Verwerken in {n_batches} batches van {batch_height} rijen...")
        for i in tqdm(range(n_batches), desc="📊 Smoothing batches"):
            start_time = time.time()

            row_off = i * batch_height
            read_rows = min(batch_height + 2 * pad, height - row_off + 2 * pad)
            row_start = max(row_off - pad, 0)
            row_stop = min(row_start + read_rows, height)

            window = Window(0, row_start, width, row_stop - row_start)
            data = src.read(1, window=window)

            smoothed = generic_filter(data, mode_filter, size=kernel_size)

            # Bepaal de write-window
            write_start = pad if row_start != 0 else 0
            write_stop = -pad if row_stop != height else None
            smoothed_write = smoothed[write_start:write_stop, :]

            write_window = Window(
                0, row_off, width, smoothed_write.shape[0]
            )
            dst.write(smoothed_write.astype("uint8"), 1, window=write_window)

            batch_time = time.time() - start_time
            total_time += batch_time

        print(f"Klaar! Gesmooth raster opgeslagen als:\n{output_path}")
        print(f"Totale tijd: {total_time/60:.1f} minuten")

📂 Raster openen...
🧮 Verwerken in 39 batches van 1024 rijen...


📊 Smoothing batches: 100%|██████████| 39/39 [4:12:49<00:00, 388.97s/it]  


✅ Klaar! Gesmooth raster opgeslagen als:
../qgis/haarlem_geclassificeerd_smoothed.tif
⏱️ Totale tijd: 15170.0 seconden (252.8 minuten)


Water als klasse 3 toevoegen (water uit top10 NL)

In [ ]:
# Padinstellingen
input_path = "../qgis/haarlem_geclassificeerd_smoothed.tif"
water_mask_path = "../qgis/water_mask.tif"
output_path = "../qgis/haarlem_met_water.tif"
nodata_value = 255
batch_height = 1024  # aantal rijen per batch

# Raster openen
with rasterio.open(input_path) as src, rasterio.open(water_mask_path) as water_src:
    profile = src.profile.copy()
    profile.update({
        "compress": "DEFLATE",
        "predictor": 2,
        "zlevel": 9,
        "dtype": "uint8",
        "nodata": nodata_value
    })

    height = src.height
    width = src.width
    water_data = water_src.read(1)
    water_transform = water_src.transform

    with rasterio.open(output_path, "w", **profile) as dst:
        n_batches = int(np.ceil(height / batch_height))
        print(f"Verwerken in {n_batches} batches van {batch_height} rijen...")

        for i in tqdm(range(n_batches), desc="💧 Water-overlay batches"):
            row_start = i * batch_height
            row_end = min((i + 1) * batch_height, height)
            window = Window(col_off=0, row_off=row_start, width=width, height=row_end - row_start)

            # Lees batch uit input
            labels = src.read(1, window=window)

            # Genereer coördinaten van pixelcentra
            rows, cols = np.meshgrid(
                np.arange(row_start, row_end),
                np.arange(0, width),
                indexing="ij"
            )
            xs, ys = rasterio.transform.xy(src.transform, rows, cols, offset="center")
            xs = np.array(xs).flatten()
            ys = np.array(ys).flatten()

            # Transformeer coördinaten naar watermask indices
            water_rows, water_cols = rowcol(water_transform, xs, ys)
            water_rows = np.clip(water_rows, 0, water_data.shape[0] - 1)
            water_cols = np.clip(water_cols, 0, water_data.shape[1] - 1)

            water_vals = water_data[water_rows, water_cols].reshape(labels.shape)

            # Overschrijf met klasse 3 waar water is
            labels[water_vals == 3] = 3

            # Wegschrijven
            dst.write(labels, 1, window=window)

print(f"Klaar! Water-overlay opgeslagen naar: {output_path}")


🔄 Verwerken in 39 batches van 1024 rijen...


💧 Water-overlay batches: 100%|██████████| 39/39 [02:55<00:00,  4.50s/it]

✅ Klaar! Water-overlay opgeslagen naar: ../qgis/haarlem_met_water.tif


Raster per buurt

In [ ]:
# Instellingen
input_raster_path = "../qgis/haarlem_met_water_corrected.tif"
buurten_path = "../qgis/buurten_haarlem.gpkg"
output_folder = "../qgis/raster_per_buurt"
pixel_size = 0.25
nodata_value = 255

# Zorg dat de outputmap bestaat
os.makedirs(output_folder, exist_ok=True)

# Raster en buurten inladen
print("Raster openen")
with rasterio.open(input_raster_path) as src:
    band = src.read(1)
    transform = src.transform
    profile = src.profile

print("Buurten inladen...")
gdf = gpd.read_file(buurten_path).to_crs(profile["crs"])
buurten = gdf[~gdf.geometry.is_empty]

# Loop over buurten
print(f"Raster opslaan voor {len(buurten)} buurten")
for _, row in tqdm(buurten.iterrows(), total=len(buurten), desc="Verwerken buurten"):
    buurt_naam = row["buurtnaam"].replace(" ", "_").replace("/", "_")
    geom = row.geometry

    # Maak masker
    mask = rasterio.features.geometry_mask([geom], transform=transform, invert=True, out_shape=band.shape)

    # Extract deel van raster
    buurt_raster = np.where(mask, band, nodata_value)

    # Opslaan
    output_path = os.path.join(output_folder, f"{buurt_naam}.tif")
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(buurt_raster, 1)

print("Klaar!")


🔍 Raster openen...
🧭 Buurten inladen...
💾 Raster opslaan voor 111 buurten...


Verwerken buurten: 100%|██████████| 111/111 [15:50<00:00,  8.57s/it]

✅ Klaar!
